*Eduardus Tjitrahardja | @edutjie | 2022*

# Neural Network

## Importing Libraries

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pprint

from random import random, seed; seed(90)

## Defining Neural Network

In [4]:
def init_neural_network(num_inputs, num_hidden, num_outputs):
    network = list()

    hidden_layer = [{"weights": [random() for i in range(num_inputs + 1)]} for i in range(num_hidden)] # +1 for bias
    network.append(hidden_layer)

    output_layer = [{"weights": [random() for i in range(num_hidden + 1)]} for i in range(num_outputs)] # +1 for bias
    network.append(output_layer)

    return network

## Creating Instance

In [8]:
network = init_neural_network(2, 2, 2)
network

[[{'weights': [0.2750720593819248, 0.17597216072115396, 0.3721997388567482]},
  {'weights': [0.4391323842653869, 0.3194359068983069, 0.8749917145414905]}],
 [{'weights': [0.5321511743263649, 0.4238104857472984, 0.9610461408998433]},
  {'weights': [0.8853659998537553, 0.05661977452722089, 0.34245435563150806]}]]

## Forward Propagation

In [9]:
def activate(weights, inputs):
    activation = weights[-1] # bias
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [11]:
# activation function: sigmoid
def transfer(activation):
    return 1.0 / (1.0 + math.exp(-activation))

## Forward Process

In [13]:
def forward_propagate(network, feature):
    inputs = feature
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron["weights"], inputs)
            neuron["output"] = transfer(activation)
            new_inputs.append(neuron["output"])
        inputs = new_inputs
    return inputs

In [14]:
features = [0.98, 0.1]
forward_propagate(network, features)

[0.8385492129472636, 0.7252910000060221]

## Back Propogration

In [15]:
def transfer_derivative(output):
    return output * (1.0 - output)

In [16]:
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        
        # error for output layer
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron["weights"][j] * neuron["delta"])
                errors.append(error)
        # error for hidden layer
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron["output"])
                
        # calculate delta for each neuron
        for j in range(len(layer)):
            neuron = layer[j]
            neuron["delta"] = errors[j] * transfer_derivative(neuron["output"])

## Updating Weight

In [ ]:
def update_weigths(network, feature, lr):
    for i in range(len(network)):
        inputs = feature if i == 0 else [neuron["output"] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron["weights"][j] += lr * neuron["delta"] * inputs[j]
            neuron["weights"][-1] += lr * neuron["delta"]

## Training Weight

In [18]:
def train_network(network, features, expected, lr, epochs):
    total_loss = []
    print_every = 4
    for epoch in range(epochs):
        sum_error = 0
        for feature, expected in zip(features, expected):
            outputs = forward_propagate(network, feature)
            sum_error += sum([(expected[i] - outputs[i])**2 for i in range(len(outputs))])
            backward_propagate_error(network, expected)
            update_weigths(network, feature, lr)
        if epoch % print_every == 0:
            print(f"Epoch {epoch}, loss: {sum_error}, lr: {lr}")
        total_loss.append(sum_error)
    for layer in network:
        for neuron in layer:
            del neuron["output"]
            del neuron["delta"]
    return total_loss

## Running Network

In [19]:
def plot_network(total_loss):
    plt.plot(total_loss)
    plt.title("Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

In [ ]:
datasets = []